# Cloudformation - autoscalingv3
`Status: incomplete`

Extend v2 template by adding a bastion host, and monitoring.
- VPC `(10.0.0.0/16)`
 - IGW with route from `0.0.0.0/0`
- 3 Public Subnets
 - `10.0.1.0/24`
 - `10.0.2.0/24`
 - `10.0.3.0/24`
 - Accesible via bastion
- AutoScalingGroup
 - Min:2, Max:2
- Application ELB
 - Healthchecks
- CloudWatch

Serving a page identifying instance information over http/80

---

In [ ]:
from troposphere import Template, Ref, GetAtt, GetAZs, Base64, Join, FindInMap, Select, Output, Parameter, Tags
from troposphere.autoscaling import AutoScalingGroup, LaunchConfiguration
from troposphere.autoscaling import Tag as asTag
from troposphere.autoscaling import Tags as asTags
import troposphere.ec2 as ec2
import troposphere.elasticloadbalancingv2 as elb
import boto3
import botocore

In [ ]:
resources = set()
t = Template()
t.add_version('2010-09-09')
t.add_description('autoscaling, application elb, bastion, cloudwatch')

# common tag to apply to most resources
site_tags = Tags(Site='example.com')

# loadbalance across 2 AZs
az1 = Select('0', GetAZs(''))
az2 = Select('1', GetAZs(''))
az3 = Select('2', GetAZs(''))

# Parameters
---

In [ ]:
keyname = Parameter('keyname', 
                    Default='main', 
                    Description='Key to use for ssh access',
                    Type='String')
t.add_parameter(keyname);

# Maps

In [ ]:
# Amazon Linux AMI 2016.09.0 (HVM), SSD Volume Type
t.add_mapping('RegionMap', { 'us-east-1':      {'ami': 'ami-b73b63a0'} # N. Virginia
                           , 'us-east-2':      {'ami': 'ami-58277d3d'} # Ohio
                           , 'us-west-1':      {'ami': 'ami-23e8a343'} # N. California
                           , 'us-west-2':      {'ami': 'ami-5ec1673e'} # Oregon
                           , 'eu-west-1':      {'ami': 'ami-9398d3e0'} # Ireland
                           , 'eu-central-1':   {'ami': 'ami-f9619996'} # Frankfurt
                           , 'ap-northeast-1': {'ami': 'ami-0c11b26d'} # Tokyo
                           , 'ap-northeast-2': {'ami': 'ami-983ce8f6'} # Seoul
                           , 'ap-southeast-1': {'ami': 'ami-b953f2da'} # Singapore
                           , 'ap-southeast-2': {'ami': 'ami-db704cb8'} # Sydney
                           , 'ap-south-1':     {'ami': 'ami-34b4c05b'} # Mumbai
                           , 'sa-east-1':      {'ami': 'ami-97831ffb'} # São Paulo 
                           });

# Resources
---

## Network

### VPC

In [ ]:
vpc = ec2.VPC('exampleVpc', CidrBlock='10.0.0.0/16', Tags=site_tags)
igw = ec2.InternetGateway('igw', Tags=site_tags)
attach_igw = ec2.VPCGatewayAttachment('attachIgw', InternetGatewayId=Ref(igw), VpcId=Ref(vpc))

resources.add(vpc)
resources.add(igw)
resources.add(attach_igw)

### Subnets

Two public subnets/AZs for use with application loadbalancer.

In [ ]:
public1 = ec2.Subnet('public1', AvailabilityZone=az1, CidrBlock='10.0.1.0/24', VpcId=Ref(vpc), Tags=site_tags)
public2 = ec2.Subnet('public2', AvailabilityZone=az2, CidrBlock='10.0.2.0/24', VpcId=Ref(vpc), Tags=site_tags)
public3 = ec2.Subnet('public3', AvailabilityZone=az3, CidrBlock='10.0.3.0/24', VpcId=Ref(vpc), Tags=site_tags)
resources.add(public1)
resources.add(public2)
resources.add(public3)

### Security Groups

In [ ]:
ssh_port = 22
bastion_rules = [ec2.SecurityGroupRule(IpProtocol='tcp', CidrIp='0.0.0.0/0', FromPort=ssh_port, ToPort=ssh_port)]
bastion_sg = ec2.SecurityGroup('bastionSg',
                               GroupDescription='allow ssh traffic',
                               SecurityGroupIngress=bastion_rules,
                               VpcId=Ref(vpc),
                               Tags=site_tags)

allowed_ports = [80]
web_rules = [ec2.SecurityGroupRule(IpProtocol='tcp', CidrIp='0.0.0.0/0', FromPort=p, ToPort=p) for p in allowed_ports]
web_rules.append(ec2.SecurityGroupRule(IpProtocol='tcp', SourceSecurityGroupId=Ref(bastion_sg), FromPort=22, ToPort=22))

web_dmz_sg = ec2.SecurityGroup('webDmzSg',
                               GroupDescription='allow http traffic',
                               SecurityGroupIngress=web_rules,
                               VpcId=Ref(vpc),
                               Tags=site_tags)

resources.add(web_dmz_sg)
resources.add(bastion_sg)

### Routes

In [ ]:
route_table = ec2.RouteTable('routeTable', VpcId=Ref(vpc), Tags=site_tags)
route_to_igw = ec2.Route('routeToIgw',
                         DestinationCidrBlock='0.0.0.0/0',
                         RouteTableId=Ref(route_table),
                         GatewayId=Ref(igw))
associate_route_to_public1 = ec2.SubnetRouteTableAssociation('associateToPublic1', 
                                                             RouteTableId=Ref(route_table),
                                                             SubnetId=Ref(public1))
associate_route_to_public2 = ec2.SubnetRouteTableAssociation('associateToPublic2', 
                                                             RouteTableId=Ref(route_table),
                                                             SubnetId=Ref(public2))
associate_route_to_public3 = ec2.SubnetRouteTableAssociation('associateToPublic3', 
                                                             RouteTableId=Ref(route_table),
                                                             SubnetId=Ref(public3))
resources.add(route_table)
resources.add(route_to_igw)
resources.add(associate_route_to_public1)
resources.add(associate_route_to_public2)
resources.add(associate_route_to_public3)

## LoadBalancing

In [ ]:
web_target_group = elb.TargetGroup('webTargetGroup',
                                   Name='webNodes',
                                   Port=80,
                                   Protocol='HTTP',
                                   HealthCheckPath='/',
                                   HealthCheckPort='80',
                                   HealthCheckProtocol='HTTP',
                                   HealthCheckIntervalSeconds=60,
                                   HealthCheckTimeoutSeconds=45,
                                   HealthyThresholdCount=4,
                                   UnhealthyThresholdCount=3,
                                   Matcher=elb.Matcher(HttpCode='200'),
                                   VpcId=Ref(vpc),
                                   Tags=site_tags)

loadbalancer = elb.LoadBalancer('webElb',
                                Name='webElb',
                                Scheme='internet-facing',
                                SecurityGroups=[Ref(web_dmz_sg)],
                                Subnets=[Ref(public1), Ref(public2), Ref(public3)],
                                Tags=site_tags)

elb_listener = elb.Listener('listen80',
                            DefaultActions=[elb.Action(Type='forward', TargetGroupArn=Ref(web_target_group))],
                            LoadBalancerArn=Ref(loadbalancer),
                            Port=80,
                            Protocol='HTTP')

resources.add(elb_listener)
resources.add(loadbalancer)
resources.add(web_target_group)

## AutoScaling

In [ ]:
script = """\
#!/bin/bash
sudo yum update -y
sudo yum install httpd -y
wget https://raw.githubusercontent.com/epequeno/aws-cf/master/scripts/make_page.py
sudo python make_page.py
sudo service httpd start
"""

def make_userdata(text):
    return Base64(Join('', text.splitlines(True)))

user_data = make_userdata(script)

launch_config = LaunchConfiguration('webLaunchConfig',
                                    AssociatePublicIpAddress=True,
                                    ImageId=FindInMap('RegionMap', Ref('AWS::Region'), 'ami'),
                                    InstanceType='t2.micro',
                                    KeyName=Ref(keyname),
                                    SecurityGroups=[GetAtt(web_dmz_sg, 'GroupId')],
                                    UserData=user_data)

asg_tag = asTag('Site', 'example.com', True)
asg = AutoScalingGroup('webAsg',
                       LaunchConfigurationName=Ref(launch_config),
                       AvailabilityZones=[az1, az2, az3],
                       VPCZoneIdentifier=[Ref(public1), Ref(public2), Ref(public3)],
                       TargetGroupARNs=[Ref(web_target_group)],
                       MinSize=3,
                       MaxSize=4,
                       Tags=[asg_tag])

resources.add(launch_config)
resources.add(asg)

## Bastion

In [ ]:
bastion = ec2.Instance('bastion',
                       AvailabilityZone=az1,
                       ImageId=FindInMap('RegionMap', Ref('AWS::Region'), 'ami'),
                       InstanceType='t2.micro',
                       KeyName=Ref(keyname),
                       SecurityGroupIds=[GetAtt(bastion_sg, 'GroupId')],
                       SubnetId=Ref(public1),
                       Tags=site_tags)
bastion_public_ip = ec2.EIP('bastionEip', Domain=Ref(vpc), InstanceId=Ref(bastion))

resources.add(bastion)
resources.add(bastion_public_ip)

# Outputs

In [ ]:
elb_dns_name = Output('elbDnsName',
                      Description='DNS name for elb',
                      Value=GetAtt(loadbalancer, 'DNSName'))
bastion_public_ip = Output('bastionPublicIp',
                           Description='public IP for bastion',
                           Value=GetAtt(bastion, 'PublicIp'))
t.add_output(elb_dns_name)
t.add_output(bastion_public_ip);

#  Build
---

Write the template to file and validate it.

In [ ]:
file_name = 'asgv3'

def build(filename):
    fname = 'outputs/{}.json'.format(filename)
    for r in resources:
        t.add_resource(r)
    
    with open(fname, 'w') as fd:
        fd.write(t.to_json())
    
    !aws cloudformation validate-template --template-body file://{fname} > /dev/null && echo "build ok"

build(file_name)

Create the stack

In [ ]:
cf_client = boto3.client('cloudformation')
stack_name = file_name
stack_body = t.to_json()
cs = cf_client.create_stack(StackName=stack_name, 
                       TemplateBody=stack_body,
                       Parameters=[{'ParameterKey': 'keyname',
                                    'ParameterValue': 'jupiter',
                                    'UsePreviousValue': False}])
print(cs['ResponseMetadata']['HTTPStatusCode'])

Check for status/outputs

In [ ]:
try:
    stack = cf_client.describe_stacks(StackName=stack_name)['Stacks'][0]
    if stack['StackStatus'] == 'CREATE_COMPLETE':
        for o in stack['Outputs']:
            print(o['OutputValue'])
    else:
        print(stack['StackStatus'])
except botocore.exceptions.ClientError:
    print("stack doesn't exist")

Delete the stack when done

In [ ]:
ds = cf_client.delete_stack(StackName=stack_name)
print(ds['ResponseMetadata']['HTTPStatusCode'])